In [15]:
!pip install pymorphy3

In [16]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

In [17]:
word = "кошкам"
parsed = morph.parse(word)[0]  # Берем первый (наиболее вероятный) разбор

print(f"Слово: {word}")
print(f"Нормальная форма: {parsed.normal_form}")
print(f"Часть речи: {parsed.tag.POS}")
print(f"Падеж: {parsed.tag.case}")
print(f"Число: {parsed.tag.number}")
print(f"Все грамматические признаки: {parsed.tag}")

Слово: кошкам
Нормальная форма: кошка
Часть речи: NOUN
Падеж: datv
Число: plur
Все грамматические признаки: NOUN,anim,femn plur,datv


<h2>Изменение падежей</h2>

In [18]:
def change_case(word, target_case):
    """Изменяет падеж слова"""
    parsed = morph.parse(word)[0]
    return parsed.inflect({target_case}).word if parsed.inflect({target_case}) else word

words = ["кошка", "стол", "красивый", "быстро"]

for word in words:
    print(f"{word} ->:")
    for case in ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']:
        new_form = change_case(word, case)
        print(f"  {case}: {new_form}")
    print()

кошка ->:
  nomn: кошка
  gent: кошки
  datv: кошке
  accs: кошку
  ablt: кошкой
  loct: кошке

стол ->:
  nomn: стол
  gent: стола
  datv: столу
  accs: стол
  ablt: столом
  loct: столе

красивый ->:
  nomn: красивый
  gent: красивого
  datv: красивому
  accs: красивого
  ablt: красивым
  loct: красивом

быстро ->:
  nomn: быстро
  gent: быстро
  datv: быстро
  accs: быстро
  ablt: быстро
  loct: быстро



In [19]:
parsed = morph.parse("делаю")[0]
parsed.tag

OpencorporaTag('VERB,impf,tran sing,1per,pres,indc')

<h2>Изменение числа</h2>

In [20]:
def change_number(word, target_number):
    """Изменяет число слова"""
    parsed = morph.parse(word)[0]
    return parsed.inflect({target_number}).word if parsed.inflect({target_number}) else word

words = ["кошка", "стол", "красивый", "бежал"]

for word in words:
    singular = change_number(word, 'sing')
    plural = change_number(word, 'plur')
    print(f"{word} -> ед.ч.: {singular}, мн.ч.: {plural}")

кошка -> ед.ч.: кошка, мн.ч.: кошки
стол -> ед.ч.: стол, мн.ч.: столы
красивый -> ед.ч.: красивый, мн.ч.: красивые
бежал -> ед.ч.: бежал, мн.ч.: бежали


<h2>Комбинированное изменение форм</h2>

In [21]:
def inflect_word(word, **grammemes):
    """Изменяет слово по заданным грамматическим признакам"""
    parsed = morph.parse(word)[0]
    return parsed.inflect(set(grammemes.values())).word if parsed.inflect(set(grammemes.values())) else word

word = "интересная"
print(f"Исходное: {word}")

# Изменяем падеж и число
forms = [
    {'case': 'gent', 'number': 'sing'},  # родительный падеж, ед.ч.
    {'case': 'datv', 'number': 'plur'},  # дательный падеж, мн.ч.
    {'case': 'ablt', 'number': 'sing'},  # творительный падеж, ед.ч.
]

for gram_dict in forms:
    new_form = inflect_word(word, **gram_dict)
    print(f"{gram_dict} -> {new_form}")

Исходное: интересная
{'case': 'gent', 'number': 'sing'} -> интересной
{'case': 'datv', 'number': 'plur'} -> интересным
{'case': 'ablt', 'number': 'sing'} -> интересной


<h2>Изменение глаголов</h2>

In [22]:
def conjugate_verb(verb, **grammemes):
    """Спрягает глагол"""
    parsed = morph.parse(verb)[0]
    return parsed.inflect(set(grammemes.values())).word if parsed.inflect(set(grammemes.values())) else verb

verb = "делать"
print(f"Глагол: {verb}")

# Изменяем время, лицо, число
forms = [
    {'number': 'sing', 'person': '1per'},  # я делаю
    {'number': 'sing', 'person': '2per'},  # ты делаешь
    {'number': 'sing', 'person': '3per'},  # он делает
    {'number': 'plur', 'person': '1per'},  # мы делаем
    {'number': 'plur', 'person': '2per'},  # вы делаете
    {'number': 'plur', 'person': '3per'},  # они делают
]

for gram_dict in forms:
    new_form = conjugate_verb(verb, **gram_dict)
    print(f"{gram_dict} -> {new_form}")

Глагол: делать
{'number': 'sing', 'person': '1per'} -> делаю
{'number': 'sing', 'person': '2per'} -> делаешь
{'number': 'sing', 'person': '3per'} -> делает
{'number': 'plur', 'person': '1per'} -> делаем
{'number': 'plur', 'person': '2per'} -> делаете
{'number': 'plur', 'person': '3per'} -> делают


Практический пример: склонение ФИО

In [23]:
def decline_name(full_name, case='gent'):
    """Склоняет ФИО в нужный падеж"""
    parts = full_name.split()
    declined_parts = []

    for part in parts:
        parsed = morph.parse(part)[0]
        declined = parsed.inflect({case}).word if parsed.inflect({case}) else part
        declined_parts.append(declined)

    return ' '.join(declined_parts)

name = "Иванов Иван Иванович"
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
case_names = ['именительный', 'родительный', 'дательный', 'винительный', 'творительный', 'предложный']

for case, case_name in zip(cases, case_names):
    declined = decline_name(name, case)
    print(f"{case_name:15} -> {declined}")

именительный    -> иванов иван иванович
родительный     -> иванова ивана ивановича
дательный       -> иванову ивану ивановичу
винительный     -> иванова ивана ивановича
творительный    -> ивановым иваном ивановичем
предложный      -> иванове иване ивановиче


Получение возможных грамматических форм и проверка возможности форм


In [24]:
def get_word_forms(word):
    """Получает все возможные формы слова"""
    parsed = morph.parse(word)[0]
    forms = set()

    if hasattr(parsed, 'lexeme'):
        for form in parsed.lexeme:
            forms.add(form.word)

    return sorted(forms)

def is_grammatical_form_possible(word, **grammemes):
    """Проверяет, возможна ли данная грамматическая форма"""
    parsed = morph.parse(word)[0]
    return parsed.inflect(set(grammemes.values())) is not None

word = "бежать"
forms = get_word_forms(word)
print(f"Все формы слова '{word}':")
for form in forms[:10]:  # Покажем первые 10 форм
    print(f"  {form}")

# Проверка возможности формы
test_cases = [
    {'number': 'plur', 'person': '1per'},
    {'gender': 'femn', 'tense': 'past'},
    {'case': 'gent', 'number': 'sing'}
]

for gram_dict in test_cases:
    possible = is_grammatical_form_possible(word, **gram_dict)
    print(f"Форма {gram_dict} возможна: {possible}")

Все формы слова 'бежать':
  беги
  бегите
  бегу
  бегут
  бегущая
  бегущего
  бегущее
  бегущей
  бегущем
  бегущему
Форма {'number': 'plur', 'person': '1per'} возможна: True
Форма {'gender': 'femn', 'tense': 'past'} возможна: True
Форма {'case': 'gent', 'number': 'sing'} возможна: True


Согласование слов с числительными

In [25]:
butyavka = morph.parse('бутявка')[0]
nekuzyavaya = morph.parse('некузявая')[0]

print(f"1 {nekuzyavaya.make_agree_with_number(1).word} {butyavka.make_agree_with_number(1).word}")
print(f"2 {nekuzyavaya.make_agree_with_number(2).word} {butyavka.make_agree_with_number(2).word}")
print(f"6 {nekuzyavaya.make_agree_with_number(6).word} {butyavka.make_agree_with_number(6).word}")
print(f"121 {nekuzyavaya.make_agree_with_number(121).word} {butyavka.make_agree_with_number(121).word}")

print(f"12 {morph.parse('бятые')[0].make_agree_with_number(6).word} {morph.parse('пуськи')[0].make_agree_with_number(12).word}")


1 некузявая бутявка
2 некузявые бутявки
6 некузявых бутявок
121 некузявая бутявка
12 бятых пусек


Задача 1.

Напишите функцию, которая определяет согласованы ли прилагательное и существительное.

In [29]:
def check_if_match_adjective_and_noun(adj, noun):
  morph = pymorphy3.MorphAnalyzer()

  adj_parsed = morph.parse(adj)[0]
  noun_parsed = morph.parse(noun)[0]

  if 'ADJF' not in adj_parsed.tag and "ADJS" not in adj_parsed.tag:
    return False

  if "NOUN" not in noun_parsed.tag:
    return False

  if_gender_match = (adj_parsed.tag.gender == noun_parsed.tag.gender)
  if_case_match = (adj_parsed.tag.case == noun_parsed.tag.case)
  if_number_match = (adj_parsed.tag.number == noun_parsed.tag.number)

  return if_gender_match & if_case_match & if_number_match

In [32]:
check_if_match_adjective_and_noun("красивые", "котенок")

False

Задача 2.

Напишите функцию, которая согласует прилагательное с существительным.



In [60]:
def match_adjective_and_noun(adj, noun):
  if check_if_match_adjective_and_noun(adj, noun):
    return (adj, noun)

  morph = pymorphy3.MorphAnalyzer()

  adj_parsed = morph.parse(adj)[0]
  noun_parsed = morph.parse(noun)[0]

  if noun_parsed.tag.number in "plur":
    noun_features = {noun_parsed.tag.case, noun_parsed.tag.number}
  else:
    noun_features = {noun_parsed.tag.gender, noun_parsed.tag.case, noun_parsed.tag.number}

  inflected_adj = adj_parsed.inflect(noun_features)[0]

  return (inflected_adj, noun)


In [62]:
(inflected_adj, noun) = match_adjective_and_noun("красивых", "котенка")
print(f"{inflected_adj} {noun}")

красивого котенка


Задача 3.

Вычислите "расстояние" между словами на основе их грамматических характеристик - количество отличающихся грамматических характеристик, деленное на общее число характеристик.

In [96]:
def get_grammatical_distance(word1, word2):
  morph = pymorphy3.MorphAnalyzer()

  word1_parsed = morph.parse(word1)[0].tag
  word2_parsed = morph.parse(word2)[0].tag

  features = {'POS', 'case', 'number', 'gender', 'person', 'animacy', 'mood',
              'tense', 'transitivity', 'voice'}

  word1_attrs = set()
  word2_attrs = set()

  for feature in features:
    if hasattr(word1_parsed, feature) and getattr(word1_parsed, feature) != None:
      word1_attrs.add(feature)

    if hasattr(word2_parsed, feature) and getattr(word2_parsed, feature) != None:
      word2_attrs.add(feature)

  common_attr = word1_attrs.intersection(word2_attrs)

  distance = 0

  for attr in common_attr:
    print(f"{getattr(word1_parsed, attr)} {getattr(word2_parsed, attr)}")
    if not getattr(word1_parsed, attr) == getattr(word2_parsed, attr):
      distance = distance + 1

  return distance

In [97]:
distance = get_grammatical_distance("делали", "котенок")
print("Расстояние: ", distance)

plur sing
VERB NOUN
Расстояние:  2
